In [ ]:
import numpy as np, matplotlib.pyplot as plt
# from importlib import reload
import cvflow as cf
% matplotlib inline

In [ ]:
import laneFindingPipeline, utils

In [ ]:
show = cf.misc.show

In [ ]:
colorFilter = cf.ComplexPipeline()

In [ ]:
laneFinder = laneFindingPipeline.LaneFinder(colorFilter=colorFilter)

In [ ]:
%%time
allFrames = laneFindingPipeline.utils.loadFrames()

In [ ]:
frame = allFrames['project'][0]

In [ ]:
show(laneFinder.draw(frame));

In [ ]:
# %%time
# figs = colorFilter.showMembers(subplotKwargs=dict(figsize=(16,9)), wspace=0)
# for ext in ('png',):
#     for fig in figs:
#         label = fig.texts[0].get_text()
#         ok = 'abcdefghijklmnopqrstuvwxyz'
#         ok += ok.upper() + '01234567890-'
#         for ch in label:
#             if ch not in ok:
#                 label.replace(ch, '_')

In [ ]:
colorFilter.draw(savePath='/tmp/test', format='pdf');

Measure frames per second without plotting.

In [ ]:
%%time
import tqdm
for frame in tqdm.tqdm_notebook(allFrames['project'][:100], unit='frame'):
    laneFinder(frame)

In [ ]:
class VisualizeFilter:
    def __call__(self, frame, clearAxes=True, closeFigure=False):
        colorFilter(frame)
        axes = getattr(self, 'axes', None)
        fig = colorFilter.showMembers(
            axes=axes, subplotKwargs=dict(figsize=(16,9)), 
            wspace=0, showMultistepParents=False
        )[0]
        self.axes = fig.axes
        for ax in fig.axes:
            cf.misc.clearTicks(ax)
        out = utils.fig2img(fig)
        if clearAxes:
            for ax in fig.axes:
                ax.cla()
        if closeFigure:
            plt.close(fig)
        return out
    
    @property
    def fig(self):
        return self.axes[0].figure
    
    def visualizeFromKey(self, k, maxFrames=np.inf):
        frames = allFrames[k]
        fpath = 'cf-%s-%s-vis' % (colorFilter, k)
        if maxFrames < np.inf:
            frames = frames[:maxFrames]
            fpath += '-%dframes' % maxFrames
        fpath += '.mp4'
        utils.transformVideo(frames, fpath, visualizePipeline, desc=fpath)
        self(frames[-1], clearAxes=False)
        
    def __del__(self):
        if hasattr(self, 'axes'):
            plt.close(self.fig)

visualizeFilter = VisualizeFilter()

In [ ]:
visualizeFilter.visualizeFromKey('project', maxFrames=2)

In [ ]:
for k in allFrames.keys():
    visualizeFilter.visualizeFromKey(k, maxFrames=32)

In [ ]:
%%time
vids = {}
for k in allFrames.keys():
    vids[k] = []
    vids[k].append(utils.transformVideo(allFrames[k], 'cf-%s-%s.mp4' % (laneFinder.colorFilter, k), laneFinder.draw, desc=k))
    vids[k].append(utils.transformVideo(allFrames[k], 'cf-%s-%s-vis.mp4' % (colorFilter, k), visualizePipeline), desc=k)

In [ ]:
import gc; gc.collect()

In [ ]:
plt.close('all')

In [ ]:
import gc; gc.collect()

In [ ]:
v1

In [ ]:
v1, v2 = vids[k]

In [ ]:
v1

In [ ]:
v2